In [3]:
#Importing required packages
import numpy as np
import matplotlib.pyplot as plt
from springrank import SpringRank
import polars as pl
import networkx as nx
from src import rank_swirl #this will only work if your ipynb file lives outside of the code folder
import io 
from PIL import Image
import cairosvg 

ModuleNotFoundError: No module named 'src'

In [ ]:
#read in df
df = pl.read_csv("data/aggXquarter.txt", separator = "\t")

#filter dfs by group + quarter
df_q1_g1 = df.filter(pl.col("study.quarter") == 1).filter(pl.col("group") == "G1")
df_q1_g2 = df.filter(pl.col("study.quarter") == 1).filter(pl.col("group") == "G2")
df_q2_g1 = df.filter(pl.col("study.quarter") == 2).filter(pl.col("group") == "G1")
df_q2_g2 = df.filter(pl.col("study.quarter") == 2).filter(pl.col("group") == "G2")
df_q3_g1 = df.filter(pl.col("study.quarter") == 3).filter(pl.col("group") == "G1")
df_q3_g2 = df.filter(pl.col("study.quarter") == 3).filter(pl.col("group") == "G2")
df_q4_g1 = df.filter(pl.col("study.quarter") == 4).filter(pl.col("group") == "G1")
df_q4_g2 = df.filter(pl.col("study.quarter") == 4).filter(pl.col("group") == "G2")

#create list / dictionary to filter by group + quarter (tuple) 
df_dict = {"Q1G1": df_q1_g1, "Q2G1": df_q2_g1, "Q3G1":df_q3_g1, "Q4G1":df_q4_g1, "Q1G2": df_q1_g2, "Q2G2": df_q2_g2, "Q3G2":df_q3_g2,"Q4G2":df_q4_g2}

In [2]:
#Attempting to normalize spring rank values
#testing one df at a time 
#using a little for loop to scale each rank individually, which also does not work
G = nx.DiGraph()
G.add_weighted_edges_from(df_dict["Q1G1"][["actor", "target","number.wins"]].rows())
nodes = list(G.nodes())
node_dict = {i: n for i, n in enumerate(nodes)}
A = nx.to_numpy_array(G, nodelist = nodes)
A_dict["Q1G1"] = A
G_dict["Q1G1"] = G
model = SpringRank(alpha = 0.1)
model.fit(A)
S = model.ranks
scaled_ranks = model.get_scaled_ranks(scale=0.75) #(this line gives the error that model does not have that attribute)
scaled = []
for rank in S:
    scaled_rank = (rank - np.min(S)) / (np.max(S) - np.min(S))
    scaled.append(scaled_rank)
# np.argsort(S)[::-1], S #create ordinal ranking
S_dict["Q1G1"] = scaled
rank_dict["Q1G1"] = {node_dict[i]: float(scaled[i]) for i in range(len(scaled)) }
#model = SpringRank(alpha = 0.1)
#S = model.ranks 
#range = np.max(S) - np.min(S)
#scaled = (S-np.min(s)) / range

NameError: name 'nx' is not defined

In [ ]:
#build dictionaries of graphs, adjacency matrices, and spring ranks, trying to make sure the nodes stay in the same spot

G_dict = {}
A_dict = {}
S_dict = {}
rank_dict = {}
for name in df_dict: 
    G = nx.DiGraph()
    G.add_weighted_edges_from(df_dict[name]["actor", "target","number.wins"].rows())
    nodes = list(G.nodes())
    node_dict = {i: n for i, n in enumerate(G.nodes())}
    A = nx.to_numpy_array(G, nodelist = nodes)
    A_dict[name] = A
    G_dict[name] = G
    model = SpringRank(alpha = 0.1)
    model.fit(A)
    S = model.ranks
    scaled_ranks = model.get_scaled_ranks(0.75)
    scaled = (S - np.min(S)) / (np.max(S) - np.min(S))
    # np.argsort(S)[::-1], S #create ordinal ranking
    S_dict[name] = scaled_ranks
    rank_dict[name] = {node_dict[i]: float(scaled[i]) for i in range(len(scaled))}

